## Module 1 - Recurrent Neural Networks and LSTM
## Objective 01 - describe neural networks used for modeling sequences

In [4]:
# imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# instantiate the layers
model = keras.Sequential()
model.add(layers.Embedding(input_dim=10000, output_dim=64))

# the output of Simple RNN will be a 2D tensor of shape(batch_size, 128)
model.add(layers.SimpleRNN(128))

# Add an additional hidden layer
model.add(layers.Dense(10))

# view the architecture
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 128)               24704     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 665,994
Trainable params: 665,994
Non-trainable params: 0
_________________________________________________________________


In [5]:
# LSTM network example
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units
model.add(layers.Dense(10))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 64)          64000     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 164,106
Trainable params: 164,106
Non-trainable params: 0
_________________________________________________________________


## Objective 02 - apply a LSTM to a text generation problem using Keras

In [8]:
# load the text
import requests

url = "https://raw.githubusercontent.com/LambdaSchool/data-science-practice-datasets/main/unit_4/sherlock.txt"
response = requests.get(url)
text = response.text

# Strip the \r\n characters
text = text.replace('\r\n', ' ')

In [9]:
# encode Data as chars

# Find the unique characters
chars = list(set(text))

# Lookup tables
char_int = {c:i for i, c, in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

print('The number of unique characters in the text:', len(chars))

The number of unique characters in the text: 91


In [11]:
# create the sequence data
maxlen = 40
step = 5

# Encode the characters using the lookup table
encoded = [char_int[c] for c in text]

# Initialize empty lists to hold the sequence
sequences = [] # one element is 40 chars long
next_char = [] # one element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i+maxlen])

print('sequences: ', len(sequences))

sequences:  54974


In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence

# Pad sequence so all are equal
seq = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=40)

# create the x and y
import numpy as np
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

# Turn on the location (set to true) when the character is present
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1

    y[i, next_char[i]] = 1

In [17]:
# build the model: a single LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Embedding

model = Sequential()
model.add(Embedding(output_dim=64, input_dim=len(chars)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')


In [18]:
# fit the model
model.fit(seq, y, batch_size=32, epochs=5, verbose=2)

Epoch 1/5
1718/1718 - 23s - loss: 2.6080
Epoch 2/5
1718/1718 - 21s - loss: 2.2304
Epoch 3/5
1718/1718 - 23s - loss: 2.1075
Epoch 4/5
1718/1718 - 23s - loss: 2.0244
Epoch 5/5
1718/1718 - 17s - loss: 1.9627


In [ ]:
# Predict and convert text back into characters
def generate_text(model, seed, length):

    encoded = [char_int[c] for c in seed]
    generated =''
    generated += seed
    model.reset_states()

    start_index = 0

    for _ in range(length):

        sample = encoded[start_index:]